# Make a yaml metadata file

Let's make your default YAML file. You should do this once per experimental design.

Please answer the following questions:

    - Have you recorded Ephys data?
    - If you have time series data, what names do you want to give them?
    - Where do you want to save your default YAML file?

In [88]:
from pathlib import Path

recorded_ephys_data = True
time_series_names = ["Licks", "Eye-tracking"]
file_path = "my_metadata_yaml_file.yml"

The recording itself and its probe contains metadata, so let's try and get that

In [89]:
from neuroconv.datainterfaces import OpenEphysRecordingInterface
folder_path = f"data"
my_ephys_interface = OpenEphysRecordingInterface(folder_path=folder_path, stream_name="Signals CH")

We'll now attach a probe to it

In [90]:
import numpy as np
import probeinterface as pi

def tetrode_probe():

    geometry = np.array([[   0,  0], [  25,  0], [  25, 25], [  0,  25],
                         [200, 200], [225, 200], [225, 225], [200, 225],
                         [400, 400], [425, 400], [425, 425], [400, 425],
                         [600, 600], [625, 600], [625, 625], [600, 625]])
    
    probe = pi.Probe(ndim=2, si_units='um')
    probe.set_contacts(positions=geometry, shapes='circle', shape_params={'radius': 5})
    probe.set_device_channel_indices(np.arange(16))
    probe.set_shank_ids(shank_ids=[0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3])

    return probe

probe = tetrode_probe()

my_ephys_interface.recording_extractor.set_probe(probe, group_mode="by_shank", in_place=True)

OpenEphysLegacyRecordingExtractor: 16 channels - 30.0kHz - 1 segments - 67,260,416 samples 
                                   2,242.01s (37.37 minutes) - int16 dtype - 2.00 GiB

That's all we need to do to create the probe. Let's extract all the metadata we can from it, and combine it with our desired TimeSeries stuff.

In [91]:
ephys_metadata = my_ephys_interface.get_metadata()

for name in time_series_names:

    ephys_metadata[name] = {"name": name, 
                        "description": '',
                        "comments": '',
                        "unit": '',
                        "resolution": 0.0,
                        "conversion": 0.0,
                        "offset": 0.0 }

If you want to combine this metadata file with another one, now's the time to do it. You can merge it with your existing ymal file as follows.

In [93]:
from neuroconv.utils.dict import load_dict_from_file
metadata_path = "basic_metadata.yml"
existing_metadata_from_file = load_dict_from_file(file_path=metadata_path)

all_metadata = dict_deep_update(existing_metadata_from_file, ephys_metadata)
all_metadata['NWBFile']['identifier'] = ephys_metadata['NWBFile']['identifier']

And export the whole metadata file:

In [ ]:
with open(file_path, 'w') as outfile:
    yaml.dump(all_metadata, outfile, default_flow_style=False, sort_keys=False)

This file will be the starting point for each session. Go explore it, and update it as needed.
Once it's ready, save it somewhere. It's your starting point that your conversion script will use.